# Calculo dos Resultados dos Clientes Conta Única

## 0 - Setup  

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:.3f}')
pd.set_option('display.max_colwidth', 100)

import numpy as np 
from matplotlib import pyplot as plt

%pip install -r requirements.txt
from google.cloud import bigquery
pmc = "`lemoncake-prod.serving_layer.modelo_para_calculos_finais_controle_recebimentos_join_cobrancas_dynamo`"
project_id = 'lemoncake-prod'
client = bigquery.Client(project = project_id)

%pip install plotly==5.17.0
import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime, timedelta

ERROR: Ignored the following versions that require a different python version: 2.10.0 Requires-Python >=3.6, <3.10; 2.11.0 Requires-Python >=3.6, <3.10; 2.12.0 Requires-Python >=3.6, <3.10; 2.13.0 Requires-Python >=3.6, <3.10; 2.13.1 Requires-Python >=3.6, <3.10; 2.14.0 Requires-Python >=3.6, <3.10; 2.15.0 Requires-Python >=3.6, <3.10; 2.16.0 Requires-Python >=3.6, <3.10; 2.16.1 Requires-Python >=3.6, <3.10; 2.17.0 Requires-Python >=3.6, <3.10; 2.18.0 Requires-Python >=3.6, <3.10; 2.19.0 Requires-Python >=3.6, <3.10; 2.20.0 Requires-Python >=3.6, <3.10; 2.21.0 Requires-Python >=3.6, <3.10; 2.22.0 Requires-Python >=3.6, <3.10; 2.22.1 Requires-Python >=3.6, <3.10; 2.23.0 Requires-Python >=3.6, <3.10; 2.23.1 Requires-Python >=3.6, <3.10; 2.23.2 Requires-Python >=3.6, <3.10; 2.23.3 Requires-Python >=3.6, <3.10; 2.24.0 Requires-Python >=3.6, <3.10; 2.24.1 Requires-Python >=3.6, <3.10; 2.25.0 Requires-Python >=3.6, <3.10; 2.25.1 Requires-Python >=3.6, <3.10; 2.25.2 Requires-Python >=3.6, <3.

Note: you may need to restart the kernel to use updated packages.


c:\Users\DELL\anaconda3\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Note: you may need to restart the kernel to use updated packages.


## 1 - Lendo os dados 

In [18]:
clientes_teste_filename = 'clientes_teste.csv'
clientes_gcontrole_filename = 'clientes_grupo_controle.csv'
folder_name = 'clientes'

teste_path = "\\".join([folder_name,clientes_teste_filename])
gcontrole_path = '\\'.join([folder_name,clientes_gcontrole_filename])

clientes_teste_df = pd.read_csv(teste_path)
n_clientes = clientes_teste_df.shape[0]

clientes_gcontrole_raw = pd.read_csv(teste_path)
clientes_gcontrole_df = clientes_gcontrole_raw.sample(n_clientes, random_state=42)

## 2 - Calculo dos resultados

1 - taxa total de pagamento em dia teste x controle
    
1 - Evolutivo do pagamento em dia por dia (inicio em 01/11/2023) 
    1 gráfico, duas linhas, teste e controle

3 - Frequencia e Severidade
    frequência de 3 meses antes da conta única - teste
    frequência de 3 meses antes da conta única - controle
    frequência atualmente (mesmo com 1 conta) - teste
    frequência atualmente (mesmo com 1 conta) - controle

    severidade de 3 meses antes da conta única - teste
    severidade de 3 meses antes da conta única - controle
    severidade de 3 atualmente (mesmo com 1 conta) - teste
    severidade de 3 atualmente (mesmo com 1 conta) - controle

    comparativo das médias da frequencia teste x controle
    comparativo das médias da frequencia teste x controle 

    Houve clientes que saíram do perfim de mal pagador? (se comparado com quando selecionamos ele)
    se sim, quantos? 
